In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

import numpy as np

import gym

tf.random.set_seed(42)
np.random.seed(42)

2023-04-12 20:16:38.181683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 20:16:43.009672: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/lib64
2023-04-12 20:16:43.009776: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-12 20:17:00.496716: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object fi

# Objective: Create a DDPG algorithm with a GPT as the Actor network.   

In [2]:
#Ornstein-Uhlenbeck Noise 
class OUActionNoise(object):
    def __init__(self, mean, sigma=0.5, theta=0.2, dt=0.3, x0=None):
        self.mean = mean
        self.sigma = sigma
        self.theta = theta
        self.dt = dt
        self.x0 = x0
        self.reset()
    
    #--------------------------------------------------------------------------------
    #Method that enables to write classes where the instances behave like functions and can be called like a function.    
    def __call__(self):
        x = self.x_prev + self.theta * (self.mean - self.x_prev) * self.dt + self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        self.x_prev = x
        
        return x
    
    #--------------------------------------------------------------------------------
    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mean)
        return

In [3]:
%%script false --no-raise-error

a = np.zeros(15)
b = OUActionNoise(a)
a += b()
a

In [4]:
#Replay Buffer 
class ReplayBuffer(object):
    def __init__(self, size, batch_size, block_size):
        '''
        Args:
            size (integer): The size of the replay buffer.              
            batch_size (integer): The batch size.
            block_size (integer): 
        '''
        self.buffer = []
        self.batch_size = batch_size
        self.max_size = size
        self.block_size = block_size
        
    #--------------------------------------------------------------------------------    
    def append(self, state, action, reward, next_state, done):
        '''
        Args:
            state (Numpy array): The state.              
            action (integer): The action.
            reward (float): The reward.
            done (boolen): True if the next state is a terminal state and False otherwise. Is transformed to integer so tha True = 1, False = 0
            next_state (Numpy array): The next state.           
        '''
        if self.size == self.max_size:
            del self.buffer[0]
        self.buffer.append((state, action, reward, next_state, int(done)))

    #--------------------------------------------------------------------------------
    def clear(self):
        self.buffer.clear()
    
    #--------------------------------------------------------------------------------    
    def getBlock(self):
        '''
        Returns:
            A list of transition tuples including state, action, reward, terminal, and next_state
        '''
        # gets the random indexes of the block start
        idxs = np.random.choice(np.arange(self.size - self.block_size), size=(self.batch_size,), replace=False)
        get_data = lambda i, idx: [mem[i] for mem in self.buffer[idx:idx+self.block_size]]
        
        # generate the batch by stacking the blocks
        states = np.array(np.stack([get_data(0, idx) for idx in idxs], axis=0))
        actions = np.array(np.stack([get_data(1, idx) for idx in idxs], axis=0))
        rewards = np.array(np.stack([get_data(2, idx) for idx in idxs], axis=0))
        next_states = np.array(np.stack([get_data(3, idx) for idx in idxs], axis=0))
        done = np.array(np.stack([get_data(4, idx) for idx in idxs], axis=0))

        return  states, actions, rewards, next_states, done
    
    #--------------------------------------------------------------------------------  
    @property  
    def size(self):
        '''
        Returns:
            Number of elements in the buffer
        '''
        return len(self.buffer)
    
    #--------------------------------------------------------------------------------
    @property 
    def hasMinLength(self):
        '''
        Returns:
            Boolean indicating if the memory have the minimum number of elements or not
        '''
        return (self.size >= (self.batch_size + self.block_size))
    
    #--------------------------------------------------------------------------------
    @property  
    def data(self):
        '''
        Returns:
            List with all the elements in the buffer
        '''
        return self.buffer

In [5]:
%%script false --no-raise-error

# a replay buffer with max lenght of 500, that generates a batch with 32 blocks containing 9 'memory elements' in each 
rb = ReplayBuffer(500, 32, 8)
for i in range(300):
    # single state dimension = 5
    state = np.random.rand(24)
    next_state = np.random.rand(24)
    # single action dimension = 4
    action = np.random.rand(4)
    # rewrd is number
    reward = np.random.rand()
    # done is a boolean
    done = np.random.rand() > 0.5
    rb.append(state, action, reward, next_state, done)

print(rb.size)
print(rb.hasMinLength)
print(rb.data[0])

st, act, rw, n_st, d = rb.getBlock()
print(st.shape)
print(act.shape)
print(rw.shape)
print(n_st.shape)
print(d.shape)

In [6]:
gpt_kernel_initializer = lambda: keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
gpt_bias_initializer = lambda: keras.initializers.Zeros()

In [7]:
# Individual Head of self-attention
class Head(layers.Layer):
    """ one head of self-attention """
    def __init__(self, batch_size, block_size, state_dim, head_size, dropout):
        super().__init__()
        self.batch_size = batch_size
        self.block_size = block_size
        self.state_dim = state_dim
        # key, query and value layers
        self.key = layers.Dense(units=head_size, use_bias=False, kernel_initializer=gpt_kernel_initializer())
        self.query = layers.Dense(units=head_size, use_bias=False, kernel_initializer=gpt_kernel_initializer())
        self.value = layers.Dense(units=head_size, use_bias=False, kernel_initializer=gpt_kernel_initializer())
        # dropout layer
        self.dropout = layers.Dropout(dropout)

    def call(self, x, training=False):
        B, T, C = x.shape
        if(B is None): B = self.batch_size 
        if(T is None): T = self.block_size
        if(C is None): C = self.state_dim
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities") - C**-0.5 is for normalization
        wei =  tf.matmul(q, tf.transpose(k, perm=[0, 2, 1]))  * tf.math.rsqrt(tf.cast(C, tf.float32)) # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = tf.where(tf.linalg.band_part(tf.ones((T, T)), -1, 0) == 0, tf.constant(float("-inf"), shape=(B, T, T)), wei) # (B, T, T)
        wei = tf.nn.softmax(wei, axis=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = tf.matmul(wei, v) # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [8]:
# Layer with multiple self-attention Heads for data communication 
class MultiHeadAttention(layers.Layer):
    """ multiple heads of self-attention in parallel """
    def __init__(self, batch_size, block_size, state_dim, num_heads, head_size, dropout):
        super().__init__()
        self.heads = [Head(batch_size, block_size, state_dim, head_size, dropout) for _ in range(num_heads)]
        # this linear layer is used to 'merge' the multiple heads acquired knowledge
        self.proj = layers.Dense(units=state_dim, kernel_initializer=gpt_kernel_initializer(), bias_initializer=gpt_bias_initializer())
        self.dropout = layers.Dropout(dropout)

    def call(self, x):
        # concatenate the heads outputs in the C dimension
        out =  tf.concat([h(x) for h in self.heads], axis=-1)
        # apply the projection and the dropout
        out = self.dropout(self.proj(out))
        return out

In [9]:
#Simple feed forward for data computation
class FeedForward(layers.Layer):
    def __init__(self, state_dim, dropout, last_resize=True, spread_dim=None):
        super().__init__()
        last_layer = [layers.Dense(state_dim, kernel_initializer=gpt_kernel_initializer(), bias_initializer=gpt_bias_initializer()), layers.Dropout(dropout)] if(last_resize) else []
        self.net = keras.Sequential([
            layers.Dense(spread_dim if spread_dim is not None else 4 * state_dim, kernel_initializer=gpt_kernel_initializer(), bias_initializer=gpt_bias_initializer()),
            layers.Dropout(dropout),
            *last_layer
        ])

    def call(self, x):
        return self.net(x)

In [10]:
# Block containing a multi head attention module and a feed forward linear computation
class Block(layers.Layer):
    """ Transformer block: communication followed by computation """
    def __init__(self, batch_size, block_size, state_dim, num_heads, dropout, last_resize, spread_dim):
        super().__init__()
        head_size = state_dim // num_heads # each head gets a portion of the embeddings so different relations can be learned
        self.sa = MultiHeadAttention(batch_size, block_size, state_dim, num_heads, head_size, dropout)
        self.ffwd = FeedForward(state_dim, dropout, last_resize, spread_dim)
        self.ln1 = layers.LayerNormalization()

    def call(self, x):
        # Multi head attention with layer norm
        x = x + self.sa(self.ln1(x))
        # feed forward with layer norm
        x = self.ffwd(x)
        return x

In [11]:
class GPTModel(keras.models.Model):
    def __init__(self, n_layer, batch_size, block_size, state_dim, action_dim, action_range, num_heads, dropout):
        super().__init__()
        self.block_size = block_size
        self.position_embedding_table = layers.Embedding(block_size, state_dim, input_length=None, embeddings_initializer=gpt_kernel_initializer())
        self.blocks = keras.models.Sequential([Block(batch_size, block_size, state_dim, num_heads, dropout, last_resize = (i != n_layer - 1 ),  spread_dim= 256 if (i == n_layer - 1 ) else None) for i in range(n_layer)])
        f_value = lambda x: tf.random_uniform_initializer(- 1 / np.sqrt(x), 1 / np.sqrt(x))
        self.ffw = keras.models.Sequential([
            layers.Dense(256,  activation='relu', kernel_initializer=f_value(256), bias_initializer=f_value(256)),
            layers.BatchNormalization(),
            layers.Dense(64,  activation='relu', kernel_initializer=f_value(64), bias_initializer=f_value(64)),
            layers.BatchNormalization(),
            layers.Dense(action_dim, activation='tanh', kernel_initializer=f_value(1/(0.03**2)), bias_initializer=f_value(1/(0.03**2))),
            layers.Lambda(lambda i: i * action_range, dtype='float64'),
        ])

    def call(self, inputs, training=False):
        B, T, C = inputs.shape
        if(T is None): T = self.block_size
        pos_emb = self.position_embedding_table(tf.range(tf.constant(T), dtype=tf.int32))
        x = inputs + pos_emb
        x = self.blocks(x)
        logits = self.ffw(x)
        return logits
    

    def generate(self, states):
        # crop idx to the last block_size tokens
        idx_cond = states[:, -self.block_size:, :]
        # get the predictions
        actions = self(idx_cond)
        # focus only on the last time step
        return actions

In [12]:
%%script false --no-raise-error

# hyperparameters
env = gym.make( "BipedalWalker-v3", max_episode_steps=100)
n_layer = 2
batch_size = 4 
block_size = 64
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_range = env.action_space.high
num_heads = 4
dropout = 0.1
num_epochs = 1
learning_rate = 8e-4


gpt = GPTModel(
    n_layer=n_layer,
    batch_size=batch_size, 
    block_size=block_size, 
    state_dim=state_dim, 
    action_dim=action_dim, 
    action_range=action_range,
    num_heads=num_heads,
    dropout=dropout,
)

gpt.compile(
    loss='mean_squared_error',
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
)

states = env.reset().reshape(1, 1, -1)
done = False
while not done:
    env.render()
    # Generating actions for a given group of states 
    action = gpt.generate(states)
    # Gets the last action only
    action = action[0, -1, :]
    # Apply the action in the environment
    new_state, reward, done, _ = env.step(action)
    # Append the new state to the states history
    states = tf.concat((states, new_state.reshape(1, 1, -1)), axis=1)
env.close()

_ = gpt.fit(
    x=np.random.rand(3*batch_size * block_size * state_dim).reshape(3*batch_size, block_size, state_dim), 
    y= np.random.rand(3*batch_size * block_size * action_dim).reshape(3*batch_size, block_size, action_dim),
    validation_data=(np.random.rand(1*batch_size * block_size * state_dim).reshape(1*batch_size, block_size, state_dim), np.random.rand(1*batch_size * block_size * action_dim).reshape(1*batch_size, block_size, action_dim)),
    batch_size=batch_size,
    epochs=num_epochs, 
    verbose=1,
)

In [13]:
class Actor(object):
    def __init__(self, n_layer, batch_size, block_size, state_dim, action_dim, num_heads, dropout, action_range, lr, tau):
        #Network dimensions
        self.inp_dim = state_dim
        self.action_dim = action_dim

        #Parameter that coordinates the soft updates on the target weights
        self.tau = tau

        #Generates the optimization function - used in the agent to generate gradients
        self.optimizer = keras.optimizers.Adam(lr)

        #Generates the actor model
        self.model = GPTModel(
            n_layer=n_layer,
            batch_size=batch_size, 
            block_size=block_size, 
            state_dim=state_dim, 
            action_dim=action_dim, 
            action_range=action_range,
            num_heads=num_heads,
            dropout=dropout,
        )
        self.model.build((None, None, state_dim))

        #Generates the actor target model
        self.target_model = GPTModel(
            n_layer=n_layer,
            batch_size=batch_size, 
            block_size=block_size, 
            state_dim=state_dim, 
            action_dim=action_dim, 
            action_range=action_range,
            num_heads=num_heads,
            dropout=dropout,
        )
        self.target_model.build((None, None, state_dim))

        #Set the weights to be the same in the begining
        self.target_model.set_weights(self.model.get_weights())
    
    #--------------------------------------------------------------------
    def predict(self, states):
        return self.model.generate(states)
    
    #--------------------------------------------------------------------
    def target_predict(self, states):
        return self.target_model.generate(states)
    
    #--------------------------------------------------------------------
    def act(self, states):
        action = self.predict(states)
        # Gets the last action only
        action = action[0, -1, :]
        return action

    #--------------------------------------------------------------------
    def transferWeights(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        new_weights = []
        
        for i in range(len(weights)):
            new_weights.append((self.tau * weights[i]) + ((1.0 - self.tau) * target_weights[i]))
        
        self.target_model.set_weights(new_weights)
        
    #--------------------------------------------------------------------
    def saveModel(self, path):
        self.target_model.save(path + '_actor.h5')
    
    #--------------------------------------------------------------------
    def loadModel(self, path):
        self.target_model = keras.models.load_model(path)
        self.model = keras.models.load_model(path)

In [14]:
%%script false --no-raise-error

env = gym.make( "BipedalWalker-v3", max_episode_steps=100)
n_layer = 2
batch_size = 30 
block_size = 64
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_range = env.action_space.high
num_heads = 4
dropout = 0.1
num_epochs = 1
learning_rate = 8e-4
tau = 5e-3

act = Actor(
    n_layer=n_layer,
    batch_size=batch_size, 
    block_size=block_size, 
    state_dim=state_dim, 
    action_dim=action_dim, 
    num_heads=num_heads, 
    dropout=dropout, 
    action_range=action_range, 
    lr=learning_rate, 
    tau=tau,
)

states = env.reset().reshape(1, 1, -1)
done = False
while not done:
    env.render()
    # Generating actions for a given group of states 
    action = act.act(states)
    # Apply the action in the environment
    new_state, reward, done, _ = env.step(action)
    # Append the new state to the states history
    states = tf.concat((states, new_state.reshape(1, 1, -1)), axis=1)
env.close()


In [15]:
class Critic(object):
    def __init__(self, state_dim, state_fc1_dim, state_fc2_dim, action_inp_dim, action_fc1_dim, conc_fc1_dim, conc_fc2_dim, out_dim, dropout, lr, tau):
        #Network dimensions
        self.state_dim = state_dim
        self.state_fc1_dim = state_fc1_dim
        self.state_fc2_dim = state_fc2_dim
        self.action_inp_dim = action_inp_dim
        self.action_fc1_dim = action_fc1_dim
        self.conc_fc1_dim = conc_fc1_dim
        self.conc_fc2_dim = conc_fc2_dim
        self.out_dim = out_dim
        self.dropout = dropout
        #Define the critic optimizer
        self.optimizer = keras.optimizers.Adam(lr)
        #Parameter that coordinates the soft updates on the target weights
        self.tau = tau
        #Generate the critic network
        self.model = self.buildNetwork()
        #Generate the critic target network
        self.target_model = self.buildNetwork()
        #Set the weights to be the same in the begining
        self.target_model.set_weights(self.model.get_weights())

    #--------------------------------------------------------------------
    def buildNetwork(self):
        #State input network ---------
        s_inp = layers.Input(shape=(self.state_dim, ))
        
        f1 = 1 / np.sqrt(self.state_fc1_dim)
        s_fc1 = layers.Dense(self.state_fc1_dim, activation='relu', kernel_initializer=tf.random_uniform_initializer(-f1, f1), bias_initializer=tf.random_uniform_initializer(-f1, f1), dtype='float64')(s_inp)
        s_fc1 = layers.Dropout(self.dropout)(s_fc1)
        s_norm1 = layers.BatchNormalization(dtype='float64')(s_fc1)
        
        f2 = 1 / np.sqrt(self.state_fc2_dim)
        s_fc2 = layers.Dense(self.state_fc2_dim, activation='relu', kernel_initializer=tf.random_uniform_initializer(-f2, f2), bias_initializer=tf.random_uniform_initializer(-f2, f2), dtype='float64')(s_norm1)
        s_fc2 = layers.Dropout(self.dropout)(s_fc2)
        s_norm2 = layers.BatchNormalization(dtype='float64')(s_fc2)
        
        #Action input network ---------
        a_inp = layers.Input(shape=(self.action_inp_dim, ))
        
        f1 = 1 / np.sqrt(self.action_fc1_dim)
        a_fc1 = layers.Dense(self.action_fc1_dim, activation='relu', kernel_initializer=tf.random_uniform_initializer(-f1, f1), bias_initializer=tf.random_uniform_initializer(-f1, f1), dtype='float64')(a_inp)
        a_fc1 = layers.Dropout(self.dropout)(a_fc1)
        a_norm1 = layers.BatchNormalization(dtype='float64')(a_fc1)
        
        #Concatenate the two networks ---
        c_inp = layers.Concatenate(dtype='float64')([s_norm2, a_norm1])
        
        #Creates the output network
        f1 = 1 / np.sqrt(self.conc_fc1_dim)
        c_fc1 = layers.Dense(self.conc_fc1_dim, activation='relu', kernel_initializer=tf.random_uniform_initializer(-f1, f1), bias_initializer=tf.random_uniform_initializer(-f1, f1), dtype='float64')(c_inp)
        c_fc1 = layers.Dropout(self.dropout)(c_fc1)
        c_norm1 = layers.BatchNormalization(dtype='float64')(c_fc1)
        
        f2 = 1 / np.sqrt(self.conc_fc2_dim)
        c_fc2 = layers.Dense(self.conc_fc2_dim, activation='relu', kernel_initializer=tf.random_uniform_initializer(-f2, f2), bias_initializer=tf.random_uniform_initializer(-f2, f2), dtype='float64')(c_norm1)
        c_fc2 = layers.Dropout(self.dropout)(c_fc2)
        c_norm2 = layers.BatchNormalization(dtype='float64')(c_fc2)
        
        f3 = 0.003
        out = layers.Dense(self.out_dim, activation='linear', kernel_initializer=tf.random_uniform_initializer(-f3, f3), bias_initializer=tf.random_uniform_initializer(-f3, f3), dtype='float64')(c_norm2)
        
        model = keras.Model(inputs=[s_inp, a_inp], outputs=[out])
        
        return model
    
    #--------------------------------------------------------------------
    def predict(self, states, actions):
        return self.model([states, actions], training=False)
    
    #--------------------------------------------------------------------
    def target_predict(self, states, actions):
        return self.target_model([states, actions], training=False)
    
    #--------------------------------------------------------------------
    def transferWeights(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        new_weights = []
        
        for i in range(len(weights)):
            new_weights.append((self.tau * weights[i]) + ((1.0 - self.tau) * target_weights[i]))
        
        self.target_model.set_weights(new_weights)
        
    #--------------------------------------------------------------------
    def saveModel(self, path):
        self.target_model.save(path + '_actor.h5')
    
    #--------------------------------------------------------------------
    def loadModel(self, path):
        self.target_model = keras.models.load_model(path)
        self.model = keras.models.load_model(path)

In [16]:
class DDPG_GPT_Agent(object):
    def __init__(self, a_n_layers, batch_size, block_size, state_dim, action_dim, a_n_heads, dropout, 
        action_min, action_max, memory_size, gamma, a_lr, c_lr, tau, epsilon, epsilon_decay, 
        epsilon_min, c_state_fc1_dim, c_state_fc2_dim, c_action_fc1_dim, c_conc_fc1_dim, c_conc_fc2_dim):
        
        self.batch_size = batch_size
        self.block_size = block_size
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.action_min = action_min
        self.action_max = action_max
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        #Creates the Replay Buffer
        self.memory = ReplayBuffer(memory_size, batch_size, block_size)

        #Creates the noise generator
        self.ou_noise = OUActionNoise(mean=np.zeros(action_dim))

        #Creates the actor
        self.actor = Actor(
            n_layer=a_n_layers,
            batch_size=batch_size, 
            block_size=block_size, 
            state_dim=state_dim, 
            action_dim=action_dim, 
            num_heads=a_n_heads, 
            dropout=dropout, 
            action_range=action_max, 
            lr=a_lr, 
            tau=tau,
        )
        
        #Creates the critic
        self.critic = Critic(
            state_dim=state_dim, 
            state_fc1_dim=c_state_fc1_dim, 
            state_fc2_dim=c_state_fc2_dim,
            action_inp_dim=self.action_dim, 
            action_fc1_dim=c_action_fc1_dim,
            conc_fc1_dim=c_conc_fc1_dim, 
            conc_fc2_dim=c_conc_fc2_dim, 
            out_dim=1,
            lr=c_lr, 
            tau=tau,
            dropout=dropout
        )
    
    #--------------------------------------------------------------------     
    def act(self, env):
        states = env.reset().reshape(1, 1, -1)
        done = False
        while not done:
            env.render()
            # Generating actions for a given group of states 
            action = self.policy(states, explore=False)
            # Apply the action in the environment
            new_state, reward, done, info = env.step(action)
            # Append the new state to the states history
            states = tf.concat((states, new_state.reshape(1, 1, -1)), axis=1)
        
    #-------------------------------------------------------------------- 
    def policy(self, states, explore=True):
        """ Generates an action from a group of states and add exploration """
        # gets the action
        action = self.actor.act(states)
        # takes the exploration with the epsilon probability
        if explore and np.random.rand() < self.epsilon: action += self.ou_noise()
        # clip the action to be between min and max values
        action = np.clip(action, a_min=self.action_min, a_max=self.action_max)
        action[np.isnan(action)] = 0
        return action
    
    #--------------------------------------------------------------------
    def learn(self, state, action, reward, next_state, done):
        """ Append an experience to the memory and replay memory if possible """
        learn_prob = 0.7
        if reward >= 0 or np.random.rand() < learn_prob: self.memory.append(state, action, reward, next_state, done)
        if self.memory.hasMinLength: self.replay_memory()
        return
        
    #--------------------------------------------------------------------    
    def replay_memory(self):
        """ Replay a batch of memories """

        # Get sample block from the replay buffer
        states, actions, rewards, next_states, done = self.memory.getBlock()
        
        states = tf.convert_to_tensor(states, dtype='float64')
        actions = tf.convert_to_tensor(actions, dtype='float64')
        rewards = tf.convert_to_tensor(rewards, dtype='float64')
        next_states = tf.convert_to_tensor(next_states, dtype='float64')
        done = tf.convert_to_tensor(done, dtype='float64')
        
        reshape_block_tensor = lambda tensor: tf.reshape(tensor, (-1, tensor.shape[-1]))
        reshape_simple_tensor = lambda tensor: tf.reshape(tensor, (-1,))

        #Train the critic
        with tf.GradientTape() as tape:
            # Compute the critic target values
            target_actions = self.actor.target_predict(next_states)
            # target actions = (1, T, A_C) // next_states = (1, T, S_C) - B = 1 because the Critic is an MLP so each block is treated as a batch
            # Compute the 'actual' reward by getting the expected return from the next state 
            y = reshape_simple_tensor(rewards) + self.gamma * self.critic.target_predict(reshape_block_tensor(next_states), reshape_block_tensor(target_actions)) * (1 - reshape_simple_tensor(done)) # y = (B, T, 1)
            # Predict the expected reward associated with taking the target predicted action in the state
            critic_value = self.critic.predict(reshape_block_tensor(states), reshape_block_tensor(actions))
            # Compute the critic loss 
            critic_loss = tf.math.reduce_mean(tf.math.square(y - critic_value))
            critic_grad = tape.gradient(critic_loss, self.critic.model.trainable_variables)

        self.critic.optimizer.apply_gradients(zip(critic_grad, self.critic.model.trainable_variables))
        
        #Train the actor
        with tf.GradientTape() as tape:
            acts = self.actor.predict(states)
            critic_grads = self.critic.predict(reshape_block_tensor(states), reshape_block_tensor(acts))
            #Used -mean as we want to maximize the value given by the critic for our actions
            actor_loss = -tf.math.reduce_mean(critic_grads)
            actor_grad = tape.gradient(actor_loss, self.actor.model.trainable_variables)
        
        self.actor.optimizer.apply_gradients(zip(actor_grad, self.actor.model.trainable_variables))
            
        #Update the model weights
        self.actor.transferWeights()
        self.critic.transferWeights() 
            
        #Decay the epsilon value
        if self.epsilon > self.epsilon_min: self.epsilon *= self.epsilon_decay
        #If its reach the minimum value it stops
        else: self.epsilon = self.epsilon_min

    #--------------------------------------------------------------------     
    def train(self, env, num_episodes, verbose, verbose_num, end_on_complete=False, complete_num=1, complete_value=float('inf'), act_after_batch=False):
        scores_history = []
        steps_history = []
        complete = 0
        print("BEGIN\n")
        
        for episode in range(num_episodes):
            done = False
            score, steps = 0, 0
            state = env.reset()
            states = state.reshape(1, 1, -1)
            
            while not done:
                action = self.policy(states)
                
                if verbose:
                    print("\r                                                                                                     ", end="")
                    print("\rEpisode: "+str(episode+1)+"\t Step: "+str(steps)+"\tReward: "+str(score) ,end="")
                    
                new_state, reward, done, _ = env.step(action)
                self.learn(state, action, reward, new_state, done)
                state = new_state
                states = tf.concat((states, new_state.reshape(1, 1, -1)), axis=1)
                score += reward
                steps += 1

            scores_history.append(score)
            steps_history.append(steps)
            
            #If the score is bigger or equal than the complete score it add one to the completed number
            if(score >= complete_value):
                complete += 1
                #If the flag is true the agent ends the trainig on the firs complete episode
                if end_on_complete and complete >= complete_num: break
            
            #These information are printed after each verbose_num episodes
            if((episode+1)%verbose_num == 0):
                print("\r                                                                                                          ", end="")
                print("\rEpisodes: ", episode+1, "/", num_episodes
                      , "\n\tTotal reward: ", np.mean(scores_history[-verbose_num:])
                      , "\n\tNum. steps: ", np.mean(steps_history[-verbose_num:])
                      , "\n\tCompleted: ", complete, "\n--------------------------")
                
                #If the flag is true the agent act and render the episode after each verbose_num episodes
                if act_after_batch: self.act(env)
                
                #Set the number of completed episodes on the batch to zero
                complete = 0

        print("\nFINISHED")
        
        return scores_history, steps_history
    #--------------------------------------------------------------------     
    def save(self, path):
        self.actor.saveModel(path)
        self.critic.saveModel(path)
    
    #--------------------------------------------------------------------
    def load(self, a_path, c_path):
        self.actor.loadModel(a_path)
        self.critic.loadModel(c_path)

In [17]:
env = gym.make( "BipedalWalker-v3", max_episode_steps=400)
batch_size = 8
block_size = 32
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_min = env.action_space.low
action_max = env.action_space.high
dropout = 0.05
memory_size = 500000
gamma = 0.99
epsilon = 1
epsilon_decay = 0.99999
epsilon_min = 0.5
tau = 8e-3

# Actor hyperparameter
a_n_layer = 2
a_num_heads = 2
a_learning_rate = 7e-4


# Critic hyperparameter
c_state_fc1_dim=256
c_state_fc2_dim=128
c_action_fc1_dim=64
c_conc_fc1_dim=256
c_conc_fc2_dim=64
c_learning_rate = 5e-4

agent = DDPG_GPT_Agent(
    a_n_layers=a_n_layer, 
    batch_size=batch_size, 
    block_size=block_size, 
    state_dim=state_dim, 
    action_dim=action_dim, 
    a_n_heads=a_num_heads, 
    dropout=dropout, 
    action_min=action_min, 
    action_max=action_max, 
    memory_size=memory_size, 
    gamma=gamma, 
    a_lr=a_learning_rate, 
    c_lr=c_learning_rate, 
    tau=tau, 
    epsilon=epsilon, 
    epsilon_decay=epsilon_decay, 
    epsilon_min=epsilon_min,
    c_state_fc1_dim=c_state_fc1_dim,
    c_state_fc2_dim=c_state_fc2_dim,
    c_action_fc1_dim=c_action_fc1_dim,
    c_conc_fc1_dim=c_conc_fc1_dim,
    c_conc_fc2_dim=c_conc_fc2_dim,
)

agent.act(env)

/home/user/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(
2023-04-12 20:17:20.149149: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/.local/lib/python3.10/site-packages/cv2/../../lib64::/opt/cuda/lib64
2023-04-12 20:17:20.149745: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/.local/lib/python3.10/site-packages/cv2/../../lib64::/opt/cuda/lib64
2023-04-12 20:17:20.175472: W tensorflow/compiler/

In [18]:
num_episodes = 1000
verbose = True
verbose_num = 20
act_after_batch = True

agent.train(
    env=env, 
    num_episodes=num_episodes, 
    verbose=verbose, 
    verbose_num=verbose_num,  
    act_after_batch=act_after_batch,
)
env.close()

BEGIN

Episode: 2	 Step: 107	Reward: -13.398052594497225                                                    

KeyboardInterrupt: 

In [19]:
env = gym.make( "BipedalWalker-v3", max_episode_steps=400)
agent.act(env)
env.close()